# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why? 
    
    This is a classification problem because it involves determining which of a finite number of categories(pass or fail) a student fits into based on a number of features(sex, address, age, school, etc)

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [2]:
# Import libraries
import numpy as np
import pandas as pd

from sklearn import cross_validation

In [3]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [4]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1]-1 #subtract 1 because last column is label, not feature
n_passed = 0.0
n_failed = 0.0
for i in student_data.passed:
    if i == "yes":
        n_passed = n_passed + 1
    else:
        n_failed = n_failed + 1
print n_students
print n_passed
grad_rate = (n_passed/n_students)*100
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

395
265.0
Total number of students: 395
Number of students who passed: 265.0
Number of students who failed: 130.0
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [5]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [6]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [7]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
#Implement train-test split to utilize cross validation, test on subset of training data
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_all, y_all, train_size= num_train, random_state=42)

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

##4.1 Decision Trees
    Decision trees are used for both classification and regression problems, offering a simple and easy to understand method to model data and perform predictions. This simplicity is a primary strength, along with it ease to visualize the model. Also, it doen't require extensive data preparation although its cost grows logarithmically with the number of data points.  The decision tree is a white box model, which allows easier explation of the results obtained. The model can also be validated using statistical tests, which makes it easier to determine the models reliability. However, decision trees can still create over complex, over fitting models. Also, common algorithms used in its implementation are not guaranteed to find the optimum global solution. Decision trees are used in data mining and price prediction.(1) 
    Since decision trees are able to perform the necessary classification problem and are easy to implement and interpret it seems to be a good model to test further. It handles the data categories and the relatively low number of samples means the cost to use it should be reasonable. 
    
    (1)1.10. Decision Trees — scikit-learn 0.17.1 documentation 
    http://scikit-learn.org/stable/modules/tree.html
    


##4.2 Naive Bayes
    Naive Bayes' have the ability to utilize a small amount of training data and perform very quick compared to more complicated models. The assumed independence of features allows it to scale well with increasing dimensions and perform well as long as the correct class is the more probable one. However, the assumption that each feature contributes independently to the variable being predicted and Naive Bayes' poor performance at estimation are negatives that need to be taken into account.  Naive Bayes' speed and proven record of performace in topics of text classification and spam filtering  make it another model to explore. (2)
    
    (2)1.9. Naive Bayes — scikit-learn 0.17.1 documentation
http://scikit-learn.org/stable/modules/naive_bayes.html
    

##4.3 Support Vector Machines(SVM)
    SVMs are capable of modeling high dimensional spaces, even if they out number the samples. The versatility of SVMs through the use of differing kernel functions also make them attracitve. The implementation of using support vectors to create the decision function reduces the cost of creating the model as well. These attributes lead to SVMs use in image and text classification. However, SVMs cannot provide direct probability estimates.
    SVMs are capable of performing classification problems and the data preparation performed which created many more dimensions to account for categorical variables make SVMs a good candidate.(3)
    
    (3) 1.4. Support Vector Machines — scikit-learn 0.17.1 documentation
http://scikit-learn.org/stable/modules/svm.html#scores-probabilities

In [8]:
# Train a model --Decision Tree
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn import tree
clf = tree.DecisionTreeClassifier()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002


In [9]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0


In [10]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.69918699187


In [12]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
#Training size 300
train_predict(clf, X_train[0:300], y_train[0:300], X_test, y_test)

#Training size 200
train_predict(clf, X_train[0:200], y_train[0:200], X_test, y_test)

#Training size 100
train_predict(clf, X_train[0:100], y_train[0:100], X_test, y_test)


# Note: Keep the test set constant

------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.003
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.616666666667
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.748091603053
------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 

In [15]:
# TODO: Train and predict using two other models

In [15]:
# Train a model --Naive Bayes
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# TODO: Run the helper function above for desired subsets of training data
#Training size 300
train_predict(clf, X_train[0:300], y_train[0:300], X_test, y_test)

#Training size 200
train_predict(clf, X_train[0:200], y_train[0:200], X_test, y_test)

#Training size 100
train_predict(clf, X_train[0:100], y_train[0:100], X_test, y_test)

# Note: Keep the test set constant


Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.80378250591
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.763358778626
------------------------------------------
Training set size: 300
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.80378250591
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.763358778626
------------------------------------------
Training set size: 200
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.840579710145
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.724409448819
---------------

In [18]:
# Train a model --Support Vector Machine
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn import svm
clf = svm.SVC()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# TODO: Run the helper function above for desired subsets of training data
#Training size 300
train_predict(clf, X_train[0:300], y_train[0:300], X_test, y_test)

#Training size 200
train_predict(clf, X_train[0:200], y_train[0:200], X_test, y_test)

#Training size 100
train_predict(clf, X_train[0:100], y_train[0:100], X_test, y_test)

# Note: Keep the test set constant


Training SVC...
Done!
Training time (secs): 0.008
Predicting labels using SVC...
Done!
Prediction time (secs): 0.006
F1 score for training set: 0.876068376068
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.783783783784
------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.009
Predicting labels using SVC...
Done!
Prediction time (secs): 0.006
F1 score for training set: 0.876068376068
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.783783783784
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.003
Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
F1 score for training set: 0.867924528302
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.781456953642
------------------------------------------
Training set size: 100
Training S

In [17]:
from IPython.display import display
import pandas
df = pandas.read_csv('DecisionTree_table.csv')
df

,Decision Tree,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,Training Size,300.000,200.000,100.000
1,Training Time,0.003,0.002,0.001
2,Prediction Time,0.001,0.000,0.000
3,F1 Score Training,1.000,1.000,1.000
4,F1 Score Test,0.617,0.748,0.643


In [19]:
df = pandas.read_csv('GaussianNB_table.csv')
df

,Na�ve Bayes-Gaussian,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,Training Size,300.000,200.000,100.000
1,Training Time,0.001,0.001,0.001
2,Prediction Time,0.000,0.001,0.000
3,F1 Score Training,0.804,0.841,0.847
4,F1 Score Test,0.763,0.724,0.803


In [20]:
df = pandas.read_csv('SVM_table.csv')
df

,SVM,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,Training Size,300.000,200.000,100.000
1,Training Time,0.008,0.003,0.001
2,Prediction Time,0.005,0.003,0.001
3,F1 Score Training,0.876,0.868,878.000
4,F1 Score Test,0.784,0.781,0.775


## 5. Choosing the Best Model


- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
    
    I have chosen to implement a Naive Bayes model given it has acheived the highest F1(.80) score on the test set. This demonstrates it is able to correctly classify whether a student will pass. This model also fits well with the generally low number of datapoints available to train on and is able to adapt if a feature value is missing. Naive Bayes also scales well if other factors are added in to the model as these additions will result in lower increases in time to train the model and the time it takes to perform predictions. This model is also very fast due to its simplicity so it should be able to perform well even with limited resources able to be dedicated to it in terms of computation power and time. 
 
 
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
    
    A Naive Bayes model assumes that each attribute examined, (e.g. gender, family size, location) all contribute independently to the overall probability of the students success. Using the training data, the Naive Bayes learner creates assigns a probability to each feature based on how likely it is associated with a student passing label. When given a new sample set, it combines these probabilities based on each feature value to provide the overall probability between 0-1 which is the probability of our student getting a passing label.


- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
 
 Given that Gaussian Naive Bayes does not have hyper parameters, there are no parameters to tune using grid search. 


- What is the model's final F<sub>1</sub> score?

.803

In [36]:
# TODO: Fine-tune your model and report the best F1 score
.803

0.803